In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
from urllib.parse import urlencode
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
client_id = '35ba33c2d7bd4e6197f58a98f6ac28cd'
client_secret = 'fd13b918dd374c14bbd2b9d0a43e1ff9'
redirect_uri = 'https://example.com/callback'
scope = 'user-top-read'

In [ ]:
auth_query = {
    'response_type': 'code',
    'client_id': client_id,
    'redirect_uri': redirect_uri,
    'scope': scope
}
auth_url = f"https://accounts.spotify.com/authorize?{urlencode(auth_query)}"
print("🔗 Go to this URL and authorize the app:\n", auth_url)

In [ ]:
# STEP 3 — Exchange auth code for access token
auth_code = input("Paste the code from the URL: ")

token_url = 'https://accounts.spotify.com/api/token'
payload = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri,
    'client_id': client_id,
    'client_secret': client_secret
}

response = requests.post(token_url, data=payload)
access_token = response.json().get('access_token')
print("✅ Access token received:", access_token[:30], "...")

In [ ]:
sp = spotipy.Spotify(auth=access_token)

# Test if token works
print("👤 User info:\n", sp.current_user())

In [ ]:
# STEP 5 — Fetch Top Tracks and Get Audio Features
results = sp.current_user_top_tracks(limit=5)
track_ids = [track['id'] for track in results['items']]
track_names = [track['name'] for track in results['items']]

def get_audio_features(ids):
    try:
        # Check if ids is empty
        if not ids:
            print("ℹ️ No track IDs provided to fetch audio features.")
            return []
        # Fetch audio features for the given track IDs
        features = sp.audio_features(ids)
        if not features:
            print("⚠️ Spotify API returned no audio features for the provided track IDs.")
            return []
        return features
    except Exception as e:
        # Print a more informative error message and re-raise the exception
        print(f"❌ Error fetching features: {type(e).__name__} - {e}")
        return [] # Return empty list on error

features = get_audio_features(track_ids)

# Check if features list is empty before creating DataFrame
if not features:
    print("🛑 Cannot create DataFrame as no audio features were fetched.")
else:
    # Combine with names for clarity
    df = pd.DataFrame(features)
    # Ensure 'track_name' column is added only if DataFrame is not empty
    df['track_name'] = track_names

    print("\n🎧 Top Track Audio Features:")
    # Check if the required columns exist before attempting to display
    required_cols = ['track_name', 'danceability', 'energy', 'valence', 'tempo']
    if all(col in df.columns for col in required_cols):
        display(df[required_cols])
    else:
        print(f"⚠️ DataFrame does not contain all required columns. Available columns: {df.columns.tolist()}")

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=014MozofjiDsnzyiOVnfsK,2xA7S4x2DXV3AZeeGLiasy,3sbjKDEYkBs4q9oLYV0Q88,0nj9Bq5sHDiTxSHunhgkFb,0OimeRm0wZjmRuoo7EVVjk with Params: {} returned 403 due to None


❌ Error fetching features: SpotifyException - http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=014MozofjiDsnzyiOVnfsK,2xA7S4x2DXV3AZeeGLiasy,3sbjKDEYkBs4q9oLYV0Q88,0nj9Bq5sHDiTxSHunhgkFb,0OimeRm0wZjmRuoo7EVVjk:
 None, reason: None
🛑 Cannot create DataFrame as no audio features were fetched.
